In [ ]:
import rclpy
from rclpy.node import Node
import smach
from state_machines_and_behaviour_trees import MonitorBatteryAndCollision, RotateBase, StopMotion, Rotate, BatteryStatus2bb, LaserScan2bb

import unittest
from unittest.mock import MagicMock

rclpy.init(args=None)


In [ ]:
class TestStateMachine(unittest.TestCase):
    def setUp(self):
        self.node = MagicMock()
        self.sm = smach.StateMachine(outcomes=['battery_low', 'collision_detected', 'all_safe'])
        with self.sm:
            smach.StateMachine.add('MONITOR', MonitorBatteryAndCollision(self.node),
                                   transitions={'safe': 'ROTATE_BASE', 
                                                'low_battery': 'battery_low', 
                                                'collision': 'collision_detected'})
            smach.StateMachine.add('ROTATE_BASE', RotateBase(self.node),
                                   transitions={'rotated': 'STOP_MOTION', 
                                                'failed': 'collision_detected'})
            smach.StateMachine.add('STOP_MOTION', StopMotion(self.node),
                                   transitions={'stopped': 'all_safe'})

    def test_initial_state(self):
        self.assertEqual(self.sm.get_initial_states(), ['MONITOR'])

    def test_transition_monitor_to_rotate(self):
        state = MonitorBatteryAndCollision(self.node)
        state.execute = MagicMock(return_value='safe')
        self.sm.set_initial_state(['MONITOR'])
        outcome = self.sm.execute()
        self.assertEqual(outcome, 'all_safe')

    def test_transition_monitor_to_battery(self):
        state = MonitorBatteryAndCollision(self.node)
        state.execute = MagicMock(return_value='low_battery')
        self.sm.set_initial_state(['MONITOR'])
        outcome = self.sm.execute()
        self.assertEqual(outcome, 'battery_low')

    def test_transition_monitor_to_collision(self):
        state = MonitorBatteryAndCollision(self.node)
        state.execute = MagicMock(return_value='collision')
        self.sm.set_initial_state(['MONITOR'])
        outcome = self.sm.execute()
        self.assertEqual(outcome, 'collision_detected')

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
